<h1>Semester Project for Decision Systems 2020/2021 (University of Warsaw)<h1>
    <h2>Joanna Kęczkowska<h2>

The goal of the competition is to create an efficient model for predicting whether the total 14-days sales of a particular product, offered by the Fitfood company at one of their FitBoxy locations in Poland, will exceed four pieces.

Provided data describe a short-term sales history of products at various point of sales (PoS). The target attribute will_it_sell tells if in the following 14 day period the total sales of a given product at a particular location will be at least 4 pcs. 

The data tables are provided as two CSV files with the ';' separator sign.

<h3>Data preparation<h3>

In [1]:
import pandas as pd

In [2]:
dataSet = pd.read_csv("FitFood_competition_data_training.csv", sep=';')
dataSet.shape

(531824, 97)

In [3]:
dataSet.describe()

,will_it_sell,product_id_unified,partner_product,cooking_mv,cooking_ov,vat,bialko_100,weglow_100,cukry_calk,tluszcz_nasyc_calk,...,avg_total_to_discount_lag1,avg_total_base_lag1,avg_sum_fv_lag1,avg_transaction_discount_count_lag1,roc1_lag1,rocPeriod_lag1,days_since_prev_delivery,sales_since_prev_delivery,available_products,is_delivery_day
count,531824.000000,531824.000000,531824.000000,231884.0,231884.0,531824.000000,496291.00000,496290.000000,496202.000000,496202.000000,...,5.288900e+05,528890.000000,528890.0,528890.000000,528890.000000,528890.000000,474094.000000,474094.000000,476634.000000,531823.000000
mean,0.119477,1104.984531,0.091600,1.0,1.0,6.628563,7.47106,19.206512,8.332242,3.861230,...,7.927875e-01,3.106582,0.0,0.098661,-0.001819,-0.008464,16.806237,0.351909,1.197972,0.118019
std,0.324350,55.861218,0.288461,0.0,0.0,4.417655,6.73606,15.012734,7.248779,3.268845,...,3.095363e+00,6.165410,0.0,0.292494,0.237530,0.622241,30.601578,1.024078,3.569655,0.322630
min,0.000000,1004.000000,0.000000,1.0,1.0,5.000000,0.00000,2.700000,0.200000,0.000000,...,-1.421085e-13,0.000000,0.0,0.000000,-2.772589,-3.465736,1.000000,0.000000,-24.000000,0.000000
25%,0.000000,1057.000000,0.000000,1.0,1.0,5.000000,3.20000,9.200000,3.000000,1.500000,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,1117.000000,0.000000,1.0,1.0,5.000000,6.80000,13.000000,5.900000,3.000000,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
75%,0.000000,1152.000000,0.000000,1.0,1.0,5.000000,8.90000,23.400000,12.600000,5.100000,...,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,9.000000,0.000000,2.000000,0.000000
max,1.000000,1193.000000,1.000000,1.0,1.0,23.000000,38.00000,61.300000,29.500000,14.300000,...,6.965000e+01,187.840000,0.0,3.000000,2.772589,3.465736,256.000000,31.000000,601.000000,1.000000


In [4]:
dataSet.head()

,will_it_sell,pos_id,product_id_unified,company_id,category_id,category_name,product_name,partner_product,address_city,diet,...,avg_total_to_discount_lag1,avg_total_base_lag1,avg_sum_fv_lag1,avg_transaction_discount_count_lag1,roc1_lag1,rocPeriod_lag1,days_since_prev_delivery,sales_since_prev_delivery,available_products,is_delivery_day
0,1,5cd2c7b75314a576b410fb6f,1146,5cc01f99def57b4350f57e6f,59005cd6c5c79d3575eb450d,Przekąski,"Superfood SPORT - banan, białko",1,Wrocław,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1.0
1,0,5b291c490663ab48e334abce,1136,5b9fa5e201f82e03b412bcff,5a0033206cdc0d08a6591bfb,Dania Lunch Małe,Szaszłyk z ryżem w kurkumie,0,Kraków,Dieta Samuraja,...,13.99,13.99,0.0,1.0,0.0,1.609438,NaN,NaN,NaN,0.0
2,0,5b1bb68ecaef965005d0e938,1051,5b1f8a17caef965005d12d7e,5abe0aed049e180557e22330,Sałatki,FitSalad - Sałatka z kurczakiem i grillowanymi...,0,Warszawa,--,...,0.00,0.00,0.0,0.0,0.0,0.000000,71.0,0.0,0.0,0.0
3,0,5cdbd1ba9083f77a4476ff58,1069,5d14ae03ad08444d3ee1eecd,5cd1a4d32b10792bc08dab31,Pan Pomidor - Pierogi,Pierogi ruskie,0,Wrocław,NaN,...,0.00,0.00,0.0,0.0,0.0,0.000000,2.0,0.0,0.0,0.0
4,0,5c1232411462a123065f3b58,1135,5beada3478729f2c8afeeb90,5a0033206cdc0d08a6591bfb,Dania Lunch Małe,Pikantny ryż jaśminowy smażony z jajkiem,0,Katowice,Dieta Samuraja,...,0.00,0.00,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.0


In [5]:
import numpy as np

#Getting categorical data
df_cat = dataSet.loc[:,dataSet.dtypes==np.object]
#Getting Numeric data
df_num = dataSet.loc[:,dataSet.dtypes!=np.object]

dataSet.info()

y = dataSet.pop('will_it_sell')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531824 entries, 0 to 531823
Data columns (total 97 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   will_it_sell                         531824 non-null  int64  
 1   pos_id                               531824 non-null  object 
 2   product_id_unified                   531824 non-null  int64  
 3   company_id                           531824 non-null  object 
 4   category_id                          531824 non-null  object 
 5   category_name                        531824 non-null  object 
 6   product_name                         531824 non-null  object 
 7   partner_product                      531824 non-null  int64  
 8   address_city                         531824 non-null  object 
 9   diet                                 282103 non-null  object 
 10  size                                 531824 non-null  object 
 11  cooking_time 

In [21]:
y.value_counts()

0    468283
1     63541
Name: will_it_sell, dtype: int64

**Note:** Features must be numeric with meaningful magnitude. We will cover this later.\
Now we need to split -dataSet- to training set and testing set.  If we want to (approximately) keep the proportion of y values through the training and test sets, then we need to pass stratify=y. This will enable stratified splitting:

In [6]:
from sklearn.model_selection import train_test_split

#X_train - dataset for training and tuning hyperparameters, X_test - dataset for evaluating our model
X_train, X_test, y_train, y_test = train_test_split(dataSet, y, test_size=0.2, random_state=42, stratify=y) 
#data was split in a stratified fashion

In [7]:
objlist = X_train.select_dtypes(include="object").columns
for cols in objlist:
    print('___________')
    print(X_train[cols].value_counts())

___________
596e2a5371059a6bc70a818b    2053
5b1b963bcaef965005d0e6b0    2027
5b61699404852c4704a26f28    2016
59fc5325c94b722506678bd1    2011
5b1bad72caef965005d0e891    2007
                            ... 
5dc69e723bf8f15dc97ec4de       5
5dc694443bf8f15dc97ec3c7       5
5dcd4c06dacdf109c179d79b       3
5dcbccb4aa5f032f78d77579       2
5dcd2fb58fc7d108e6b8bc09       1
Name: pos_id, Length: 375, dtype: int64
___________
5a587706cf5c8134b3a9891d    7952
5c48270855f3af637b69b13f    7559
5b9fa5e201f82e03b412bcff    6331
5b503c4a9586df16bbb6e07a    6135
5c924f69b6cb840dcac430fe    5983
                            ... 
5dbff3dd9f38f239c2ffeb0c      20
5dc534a2fdcd0622bd7b34e5      18
5e4550dd177446520bcd2c91      11
5d764e2fd0c59c62f781ff36       5
5d764e24bf7a586310594da4       4
Name: company_id, Length: 298, dtype: int64
___________
5a6f110ca0899f5ca2f7d6e9    90519
59005cd6c5c79d3575eb450d    73645
5a0033206cdc0d08a6591bfb    63665
591301c83dd75608a9c2ef1b    39234
590053bdc5c79d3575

File "encoding" contains pipelines with list of transforms for categorial variables and numeric variables.   
##### For categorial variables:  
**Simple imputer** fills missing cells with 'missing'.  
**HashEncoder** is suitable for categorial variables with large numer of levels.  
**OneHotEncoder** refers to splitting the column which contains numerical categorical data to many columns depending on the number of categories present in that column. Each column contains “0” or “1” corresponding to which column it has been placed.   Collinear features cause problems, such as when feeding the resulting data into a neural network or an unregularized regression. Therefore parameter drop='first'.

##### For numeric variables:  
**Simple imputer** fills missing cells with mode for each column.  

In [8]:
from encoding import encode

#colmun of the same value in each row is useless
del X_train["cooking_time"]
del X_test["cooking_time"]

X_train, X_test=encode(X_train, X_test)

In [18]:
sns.heatmap(X_train.corr())

NameError: name 'sns' is not defined

**Permutation Importance** 
eli5 provides a way to compute feature importances for any black-box estimator by measuring how score decreases 
when a feature is not available.

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
import lightgbm as lgbm

In [ ]:
from sklearn.feature_selection import VarianceThreshold
for col in data.columns:
if data[col].nunique()==1:
del data[col]
 
selector = VarianceThreshold(0.1)
selector.fit_transform(data)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
 
params = {
'task': 'train',
'boosting_type': 'gbdt',
'objective': 'binary',
'metric': 'auc',
'num_leaves': 18,
'learning_rate': 0.05,
'lambda_l1': 0.2,
'is_unbalance': True,
}
 
gbm = lgb.train(params, lgb_train, num_boost_round=700, verbose_eval=10)
 
#pred = gbm.predict(test_x)

In [15]:
#save the model to file
# import pickle

# filename = 'finalized_model.sav'
# pickle.dump(lgb_train, open(filename, 'wb'))

In [10]:
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)